In [1]:
! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/

In [2]:
! kaggle datasets download -d salader/dogs-vs-cats

Dataset URL: https://www.kaggle.com/datasets/salader/dogs-vs-cats
License(s): unknown


In [3]:
import zipfile
zip_ref = zipfile.ZipFile('/content/dogs-vs-cats.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [4]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten

In [5]:
# Generator

train_dataset = keras.utils.image_dataset_from_directory( # Changed keras.utis to keras.utils
    directory = '/content/train',
    image_size=(256, 256),
    batch_size=32,
    label_mode='int',
    labels='inferred',
    subset='training',
    validation_split=0.2, # Added validation_split
    seed=123
)


validation_dataset = keras.utils.image_dataset_from_directory( # Changed keras.utis to keras.utils, Changed to validation_dataset
    directory = '/content/test',
    image_size=(256, 256),
    batch_size=32,
    label_mode='int',
    labels='inferred',
    subset='validation', # Changed subset to validation
    validation_split=0.2, # Added validation_split
    seed=123
)

Found 20000 files belonging to 2 classes.
Using 16000 files for training.
Found 5000 files belonging to 2 classes.
Using 1000 files for validation.


In [6]:
def process(image, label):
  image = tf.cast(image/255. , tf.float32)
  return image, label

train_dataset = train_dataset.map(process)
validation_dataset = validation_dataset.map(process)

In [7]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), padding='valid', activation='relu', input_shape=(256, 256,3)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid'))

model.add(Conv2D(64, kernel_size=(3, 3), padding='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid'))

model.add(Conv2D(128, kernel_size=(3, 3), padding='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid'))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))



/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 254, 254, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 127, 127, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 125, 125, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 62, 62, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 60, 60, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 30, 30, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 115200)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │      14,745,728 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,847,297 (56.64 MB)

 Trainable params: 14,847,297 (56.64 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_dataset, epochs=10, validation_data=validation_dataset)

Epoch 1/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 41s 65ms/step - accuracy: 0.5716 - loss: 0.7175 - val_accuracy: 0.6640 - val_loss: 0.6263
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 32s 61ms/step - accuracy: 0.6965 - loss: 0.5761 - val_accuracy: 0.7660 - val_loss: 0.5000
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 30s 60ms/step - accuracy: 0.7773 - loss: 0.4635 - val_accuracy: 0.7360 - val_loss: 0.5477
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 32s 64ms/step - accuracy: 0.8487 - loss: 0.3405 - val_accuracy: 0.7460 - val_loss: 0.7388
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 39s 60ms/step - accuracy: 0.9195 - loss: 0.1963 - val_accuracy: 0.7690 - val_loss: 0.9120
Epoch 6/10
456/500 ━━━━━━━━━━━━━━━━━━━━ 2s 58ms/step - accuracy: 0.9572 - loss: 0.1179

In [ ]:
import cv2
test_img = cv2.imread('/content/cat1.jpeg')


In [ ]:
test_img.shape

In [ ]:
import cv2
test_img = cv2.imread('/content/cat1.jpeg')

# Resize the image to the desired dimensions before reshaping
test_img = cv2.resize(test_img, (256, 256))

test_input = test_img.reshape((1, 256, 256, 3))
test_input.shape

In [ ]:
model.predict(test_input)